In [58]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from PIL import Image
import shutil


In [59]:
# Directorio de imágenes
images_dir = 'arcgis-survey-images'

In [60]:
# Directorio para guardar el nuevo dataset
output_dir = 'processed_dataset'
train_dir = os.path.join(output_dir, 'train')
valid_dir = os.path.join(output_dir, 'valid')

In [61]:
# Crear directorios de salida si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

In [62]:
# Obtener las clases a partir de los nombres de los subdirectorios
class_names = sorted([d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))])
print(f"Clases encontradas: {class_names}")

Clases encontradas: ['Chinche salivosa', 'Clororis', 'Hoja sana', 'Roya naranja', 'Roya purpura']


In [63]:
# Crear carpetas para cada clase en los directorios de entrenamiento y validación
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, class_name), exist_ok=True)

In [64]:
# Recopilar rutas de imágenes y etiquetas
data = []
for class_label in class_names:
    class_dir = os.path.join(images_dir, class_label)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(class_dir, img_name)
            data.append({'image': img_path, 'label': class_label})

In [65]:
# Crear un DataFrame
df = pd.DataFrame(data)
print(f"Total de imágenes: {len(df)}")

Total de imágenes: 3289


In [66]:
# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=123)
print(f"Entrenamiento: {len(train_df)} imágenes")
print(f"Validación: {len(valid_df)} imágenes")

Entrenamiento: 2631 imágenes
Validación: 658 imágenes


In [67]:
# Manejo de clases desequilibradas en el conjunto de entrenamiento
class_counts = train_df['label'].value_counts()
max_count = class_counts.max()


In [68]:
# Aumentar las muestras de las clases minoritarias
dfs = []
for class_label, count in class_counts.items():
    df_class = train_df[train_df['label'] == class_label]
    if count < max_count:
        df_class = resample(df_class,
                            replace=True,     # Muestra con reemplazo
                            n_samples=max_count,  # Aumentar a max_count
                            random_state=123)
    dfs.append(df_class)


In [69]:
train_df_balanced = pd.concat(dfs)

In [70]:
# Mezclar el DataFrame balanceado
train_df_balanced = train_df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)


In [71]:
# Transformaciones para el conjunto de entrenamiento
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Lambda(lambda img: apply_sobel(np.array(img))),
    transforms.ToTensor()
])

# Transformaciones para el conjunto de validación (sin aumentación)
valid_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Lambda(lambda img: apply_sobel(np.array(img))),
    transforms.ToTensor()
])

In [72]:
# Dataset personalizado
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image']
        label = self.dataframe.iloc[idx]['label']
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        # Convertir la etiqueta a un índice
        label_idx = class_names.index(label)
        return image, label_idx, img_path

# Crear datasets y dataloaders
train_dataset = CustomDataset(train_df_balanced, transform=train_transforms)
valid_dataset = CustomDataset(valid_df, transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [73]:
# Función para guardar las imágenes procesadas
def save_processed_images(loader, save_dir):
    for images, labels, paths in loader:
        for i in range(len(images)):
            # Convertir el tensor a imagen en escala de grises
            image_np = images[i].numpy().transpose(1, 2, 0).squeeze() * 255  # Escalar a [0, 255]
            
            # Verificar si la imagen es en escala de grises y convertir a RGB
            if len(image_np.shape) == 2:  # Imagen en escala de grises
                image_pil = Image.fromarray(image_np.astype(np.uint8)).convert('RGB')
            else:
                image_pil = Image.fromarray(image_np.astype(np.uint8))
            
            # Obtener la etiqueta y el nombre original
            label = class_names[labels[i]]
            img_name = os.path.basename(paths[i])

            # Definir el path de salida
            save_path = os.path.join(save_dir, label, img_name)
            image_pil.save(save_path)



In [74]:

# Guardar el dataset de entrenamiento procesado
save_processed_images(train_loader, train_dir)



In [75]:
# Guardar el dataset de validación procesado
save_processed_images(valid_loader, valid_dir)

print("Imágenes procesadas y guardadas en el directorio:", output_dir)

Imágenes procesadas y guardadas en el directorio: processed_dataset
